In [2]:
import pandas as pd
import glob

In [59]:
def read_for_sale(files):
    all_files = []
    for f in files:
        df = pd.read_csv(f)
        df['UPDATED'] = str(f)
        all_files.append(df)
    return pd.concat(all_files)

In [60]:
# Create dataframes
all_for_sale = glob.glob("for_sale/*.csv")
all_sold = glob.glob("sold/*.csv")

df_for_sale = read_for_sale(all_for_sale)
df_sold = pd.concat(pd.read_csv(f) for f in all_sold)

In [61]:
df_for_sale.reset_index(inplace=True)
df_sold.reset_index(inplace=True)

In [62]:
# Drop the the columns we don't need
df_for_sale.drop(['index', 'SOLD DATE', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'FAVORITE', 'INTERESTED'], axis=1, inplace=True)
df_sold.drop(['index', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'FAVORITE', 'INTERESTED'], axis=1, inplace=True)

In [63]:
# Clean up Updated field
df_for_sale['UPDATED'] = df_for_sale['UPDATED'].str.slice(start=9, stop=19)
df_for_sale['UPDATED'] = pd.to_datetime(df_for_sale['UPDATED'])

In [69]:
# Get the rows where MLS# is duplicated
df_for_sale[df_for_sale.duplicated(['MLS#'])]

,index,SALE TYPE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,LATITUDE,LONGITUDE,UPDATED
250,0,PAST SALE,Townhouse,1717 A 14th Ave,Seattle,WA,98122,924900,2.0,2.0,...,1.0,779.0,NaN,Pending,https://www.redfin.com/WA/Seattle/1717-14th-Av...,NWMLS,1926225,47.616998,-122.314539,2022-05-02
352,0,MLS Listing,Single Family Residential,10423 17th Ave NE,Seattle,WA,98125,875000,3.0,2.0,...,1.0,572.0,NaN,Active,https://www.redfin.com/WA/Seattle/10423-17th-A...,NWMLS,1925732,47.704711,-122.309999,2022-05-09
353,1,MLS Listing,Single Family Residential,9209 Renton Ave S,Seattle,WA,98118,912000,5.0,2.0,...,1.0,456.0,NaN,Active,https://www.redfin.com/WA/Seattle/9209-Renton-...,NWMLS,1929665,47.520614,-122.273719,2022-05-09
354,2,MLS Listing,Single Family Residential,1107 E 72ND St,Tacoma,WA,98404,525000,4.0,2.5,...,1.0,298.0,NaN,Active,https://www.redfin.com/WA/Tacoma/1107-E-72nd-S...,NWMLS,1914014,47.192003,-122.414925,2022-05-09
355,3,MLS Listing,Single Family Residential,910 Martin Luther King Jr Way,Seattle,WA,98122,800000,3.0,2.0,...,1.0,430.0,NaN,Active,https://www.redfin.com/WA/Seattle/910-Martin-L...,NWMLS,1928854,47.610408,-122.295887,2022-05-09
356,4,MLS Listing,Single Family Residential,3511 N Shirley St,Tacoma,WA,98407,550000,3.0,2.0,...,1.0,379.0,NaN,Active,https://www.redfin.com/WA/Tacoma/3511-N-Shirle...,NWMLS,1930017,47.280193,-122.510238,2022-05-09
357,5,MLS Listing,Single Family Residential,2601 N Carr St,Tacoma,WA,98403,499500,3.0,1.0,...,1.0,223.0,NaN,Active,https://www.redfin.com/WA/Tacoma/2601-N-Carr-S...,NWMLS,1930081,47.271158,-122.466535,2022-05-09
534,126,MLS Listing,Single Family Residential,7315 19th Ave SW,Seattle,WA,98106,650000,3.0,1.5,...,1.0,670.0,NaN,Active,https://www.redfin.com/WA/Seattle/7315-19th-Av...,NWMLS,1926983,47.536613,-122.359138,2022-05-12
1106,0,MLS Listing,Single Family Residential,10604 9th Avenue Ct S,Tacoma,WA,98444,710000,4.0,3.0,...,1.0,270.0,NaN,Active,https://www.redfin.com/WA/Tacoma/10604-9th-Ave...,NWMLS,1927760,47.161434,-122.447335,2022-05-06
1119,13,MLS Listing,Condo/Co-op,737 Olive Way #2409,Seattle,WA,98101,652000,1.0,1.0,...,1.0,945.0,569.0,Active,https://www.redfin.com/WA/Seattle/737-Olive-Wa...,NWMLS,1926384,47.613645,-122.334230,2022-05-06


In [72]:
# Lookup a single MLS# to see the duplicates
df_for_sale[['ADDRESS', 'PRICE', 'UPDATED']].loc[df_for_sale['MLS#'] == 1929290]

,ADDRESS,PRICE,UPDATED
351,1808 Minor Ave #1310,425000,2022-05-08
1121,1808 Minor Ave #1310,425000,2022-05-06


In [74]:
# Try to merge for sale and sold to see if sold prices are captured accurately
# are there any sold above listing price?
pd.merge(df_for_sale, df_sold[['SOLD DATE', 'PRICE', 'MLS#']], how='inner', on='MLS#')

,index,SALE TYPE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE_x,BEDS,BATHS,...,HOA/MONTH,STATUS,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,LATITUDE,LONGITUDE,UPDATED,SOLD DATE,PRICE_y
0,9,MLS Listing,Single Family Residential,926 N 93rd St,Seattle,WA,98103,795000,4.0,2.0,...,NaN,Active,https://www.redfin.com/WA/Seattle/926-N-93rd-S...,NWMLS,1925670,47.696550,-122.345842,2022-04-30,May-11-2022,800000
1,63,MLS Listing,Single Family Residential,4826 S C St,Tacoma,WA,98408,550000,4.0,2.5,...,NaN,Active,https://www.redfin.com/WA/Tacoma/4826-S-C-St-9...,NWMLS,1922100,47.212624,-122.435686,2022-04-28,May-13-2022,620000
2,72,MLS Listing,Single Family Residential,2131 S Graham St,Seattle,WA,98108,624777,4.0,2.0,...,NaN,Active,https://www.redfin.com/WA/Seattle/2131-S-Graha...,NWMLS,1924190,47.546317,-122.305299,2022-04-28,May-11-2022,610000
3,72,MLS Listing,Single Family Residential,2131 S Graham St,Seattle,WA,98108,624777,4.0,2.0,...,NaN,Active,https://www.redfin.com/WA/Seattle/2131-S-Graha...,NWMLS,1924190,47.546317,-122.305299,2022-04-28,May-11-2022,610000
4,72,MLS Listing,Single Family Residential,2131 S Graham St,Seattle,WA,98108,624777,4.0,2.0,...,NaN,Active,https://www.redfin.com/WA/Seattle/2131-S-Graha...,NWMLS,1924190,47.546317,-122.305299,2022-04-28,May-11-2022,610000
5,82,MLS Listing,Condo/Co-op,150 Valley St #402,Seattle,WA,98109,710000,2.0,2.0,...,685.0,Active,https://www.redfin.com/WA/Seattle/150-Valley-S...,NWMLS,1919916,47.626558,-122.353916,2022-04-28,May-13-2022,777000
6,134,MLS Listing,Condo/Co-op,13275 15th Ave NE Unit B-18,Seattle,WA,98125,420000,2.0,2.0,...,811.0,Active,https://www.redfin.com/WA/Seattle/13275-15th-A...,NWMLS,1911659,47.725305,-122.312784,2022-04-28,May-13-2022,555000
7,155,MLS Listing,Single Family Residential,3643 50th Ave SW,Seattle,WA,98116,746900,3.0,2.0,...,NaN,Active,https://www.redfin.com/WA/Seattle/3643-50th-Av...,NWMLS,1913361,47.570907,-122.396432,2022-04-28,May-13-2022,900000
8,231,MLS Listing,Single Family Residential,811 NE 57th St,Seattle,WA,98105,675000,2.0,1.5,...,NaN,Active,https://www.redfin.com/WA/Seattle/811-NE-57th-...,NWMLS,1908338,47.669854,-122.319549,2022-04-28,May-6-2022,810000
9,238,MLS Listing,Condo/Co-op,952 N 35th St #202,Seattle,WA,98103,420000,1.0,1.0,...,336.0,Active,https://www.redfin.com/WA/Seattle/952-N-35th-S...,NWMLS,1922724,47.649933,-122.345056,2022-04-28,May-12-2022,510000


In [ ]:
_